# Lemmatize prog and impf results

Here are the dates from the docs:
<ul>
<li>1  = 1150-1250 : mx1, m1
<li>2  = 1250-1350 : m2, m23, m24
<li>3  = 1350-1420 : m3, m34
<li>4  = 1420-1500 : mx4, m4
<li>5  = 1500-1569 : e1
<li>6  = 1570-1639 : e2
<li>7  = 1640-1710 : e3
<li>8  = 1711-1779 : various
<li>9  = 1780-1849 : various
<li>10 = 1850-1919 : various
</ul>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
prog = pd.read_csv("progwithera.csv")
prog.head()

In [ ]:
prog = prog.loc[:, ["vbg","beform","fn","era"]]

In [ ]:
vbgs = "(VBG " + prog[prog.era <= 4].vbg +")"

In [ ]:
lemmas = pd.read_csv("vbgstoparse.lemma.csv", index_col=0, header=0, names = ['lemmas'])

In [ ]:
lemm = lemmas.lemmas.str.extract(r'l=(\w+)')

In [ ]:
pd.concat([prog, lemm], axis=1, sort=False).loc[4000]

In [ ]:
prog = pd.concat([prog, lemm], axis=1, sort=False)

In [ ]:
prog.to_csv("progEL4.csv")

Now for impf

In [ ]:
impf = pd.read_csv("impfwithera-fix.csv")

impf = impf.loc[:, ["V1", "fn", "era"]]

impf.loc[impf.fn == "victoria-186x-official-letters-2", "era"] = 10

impfs = "(VBP "+impf[impf.era <= 4].V1+")"

impfs.to_csv("impfstoparse-fix.psd")

lemmasi = pd.read_csv("impfstoparse-fix.lemma.psd", index_col=0, header=0, names=['lemmas'])
lemmasi.describe()

lemmi = lemmasi.lemmas.str.extract(r'l=(\w+)')
impf = pd.concat([impf, lemmi], axis=1, sort=False)
impf.to_csv("impfEL4.csv")

In [ ]:
prog.loc[prog.fn.str.contains("victoria") & (prog.era == 10)]

In [ ]:
prog.to_csv("progEL4-fixed.csv")

In [ ]:
plemmas = prog.loc[prog.era > 6, "vbg"].apply(lambda x: lmt(x))

In [ ]:
prog = prog.rename(columns = {0: "lemma"})

In [ ]:
plemmas = pd.Series(plemmas, name="lemma")
plemmas

In [ ]:
prog2 = pd.concat([prog, plemmas.to_frame()], axis=1, sort=False)

prog[prog.era == 3]

In [ ]:
prog.to_csv("prog-mostlemmas.csv")

In [ ]:
import nltk

from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()

def lmt(word):
    return wnl.lemmatize(word, pos="v")

In [ ]:
impf = pd.read_csv("impfEL4.csv")
impf = impf.rename(columns = {"0": "lemma"})
impf = impf.loc[:, ["V1", "fn", "era", "lemma"]]
impf.head()

In [ ]:
ilemmas = impf.loc[impf.era > 6, "V1"].apply(lambda x: lmt(x))

ilemmas = pd.Series(ilemmas, name="lemma2")
ilemmas.head()

In [ ]:
impf2 = pd.concat([impf, ilemmas.to_frame()],axis=1, sort=False)
impf2.head(20)

In [ ]:
impf2.loc[pd.isna(impf2.iloc[:,3]),"lemma"] = ilemmas

In [ ]:
impf2 = impf2.loc[:, ["V1", "fn", "era", "lemma"]]
impf2.head()

In [ ]:
MElemmas = impf.loc[(impf.era<5) & (impf.lemma.notna()),['V1','lemma']]

In [ ]:
MElemmas = MElemmas.values.tolist()

In [ ]:
MElemDict = {t[0]: t[1] for t in MElemmas}
MElemDict

In [ ]:
lemsFixed = impf.loc[(impf.era<5) & (impf.lemma.isna()),'V1'].apply(lambda x: MElemDict.get(x.lower(), np.nan))
lemsFixed

<h1>Fixing higher era lemmas</h1>
<p>First, fix dollar sign tokens</p>

In [ ]:
prog = pd.read_csv("prog-mostlemmas.csv", usecols=[1,2,3,4,5])
impf = pd.read_csv("impf-mostlemmas.csv", usecols=[1,2,3,4])

In [ ]:
prog.loc[prog.vbg.str.contains("\W"),:]

In [ ]:
prog.loc[623, "lemma"] = "pack"
prog.loc[623]

In [ ]:
prog.loc[3666, "lemma"] = "lie"

In [ ]:
prog.to_csv("prog-mostlemmas2.csv")

In [ ]:
impf.V1 = impf.V1.str.lower()
impf.lemma = impf.lemma.str.lower()
impf.head(50)

In [ ]:
impfLems = impf.loc[(impf.V1.str.contains("\$") != True) & (pd.notna(impf.lemma)) & (impf.era>7), ["V1", "lemma"]]
impfLems = impfLems.values.tolist()

In [ ]:
impfLemsDict = {t[0]: t[1] for t in impfLems}

In [ ]:
def fixDollarImpf(v1):
    v1 = str(v1)
    return impfLemsDict.get(v1[1:], np.nan)

In [ ]:
dolForms = impf.loc[impf.V1.str.contains("^\$"), "V1"]
dolForms = dolForms.values.tolist()

In [ ]:
impf["lemma"] = impf.loc[impf.V1.isin(dolForms),"lemma"].apply(lambda x: fixDollarImpf(x))

In [ ]:
impf.loc[impf.V1.isin(dolForms),:]

In [ ]:
progb = pd.read_csv("progEL4-new.csv")
progb.head()

In [ ]:
progforms = progb.loc[progb.era >=5, "vbg"]
progforms2 = progforms.str.lower().unique()

In [ ]:
pd.DataFrame(progforms2).to_csv("vbgsCor.csv")

In [ ]:
impfb = pd.read_csv("impf-mostlemmas.csv")
impfb.head()

In [ ]:
impfforms = impfb.loc[impfb.era >= 5, "V1"]
impfforms2 = impfforms.str.lower().unique()

pd.DataFrame(impfforms2).to_csv("impfCor.csv")

<h1>Combining tokens and lemmas</h1>

In [2]:
import numpy as np
import pandas as pd

In [ ]:
progOld = pd.read_csv("progEL4-new.csv", usecols=[1,2,3,4,5,6])
progOld = progOld.rename(columns = {"0": "lemma"})
progOld.head()

In [ ]:
progCors = pd.read_csv("vbgsCor-done.csv", usecols=[1,2])
progCors.head()

In [ ]:
progDicts = {t[0]: t[1] for t in progCors.values.tolist()}
progDicts 

In [ ]:
progOld.loc[progOld.era>=5, "lemma"] = progOld.loc[progOld.era>=5, "vbg"].str.lower().map(progDicts)

In [ ]:
progOld.loc[74, "lemma"] = "wear"
progOld.loc[76, "lemma"] = "overwhelm"
progOld.loc[progOld.lemma.isna()]

In [ ]:
progOld.to_csv("progLemmatized.csv")

In [ ]:
impfOld = pd.read_csv("impf-mostlemmas.csv", usecols=[1, 2, 3, 4])
impfOld.head()

In [62]:
impfCors = pd.read_csv("impfCor-done.csv", usecols=[1,2,3])

In [63]:
impfCors.loc[impfCors.comments == "not vbp", "lemma"] = "NA"
impfCors.loc[(impfCors.comments.notna()) & (impfCors.lemma != "NA")]

,0,lemma,comments
2746,*,???,***
2897,vaseline,vaseline,imperative
3244,arrah,???,***
5214,ka,???,***
5377,are,be,*** not vbp
6200,$is$,be,not vbp (be)
7100,dorse,dorse,***


In [64]:
impfCors.loc[2746, "lemma"] = "NA"
impfCors.loc[2897, "lemma"] = "NA"
impfCors.loc[3244, "lemma"] = "NA"
impfCors.loc[5214, "lemma"] = "NA"
impfCors.loc[5377, "lemma"] = "NA"
impfCors.loc[6200, "lemma"] = "NA"
impfCors.loc[(impfCors.comments.notna()) & (impfCors.lemma != "NA")]

,0,lemma,comments
7100,dorse,dorse,***


In [65]:
impfDicts = {t[0]: t[1] for t in impfCors.values.tolist()}
impfDicts 

{'opens': 'open',
 'consider': 'consider',
 'remains': 'remain',
 'takes': 'take',
 'depends': 'depend',
 'nourish': 'nourish',
 'maintain': 'maintain',
 'carry': 'carry',
 'obtains': 'obtain',
 'appears': 'appear',
 'covers': 'cover',
 'says': 'say',
 'meets': 'meet',
 'assume': 'assume',
 'asserts': 'assert',
 'makes': 'make',
 'suppose': 'suppose',
 'appear': 'appear',
 'lie': 'lie',
 'learn': 'learn',
 'regenerates': 'regenerate',
 'preserves': 'preserves',
 'prevents': 'prevent',
 'run': 'run',
 'dries': 'dry',
 'shrink': 'shrink',
 'disappear': 'disappear',
 'form': 'form',
 'find': 'find',
 'contracts': 'contract',
 'gets': 'get',
 'suffices': 'suffice',
 'arrives': 'arrive',
 'grow': 'grow',
 'expands': 'expand',
 'rushes': 'rush',
 'varies': 'vary',
 'exists': 'exist',
 'forms': 'form',
 'adheres': 'adhere',
 'falls': 'fall',
 'consists': 'consist',
 'derives': 'derive',
 'depend': 'depend',
 'thinks': 'think',
 'leaves': 'leave',
 'adhere': 'adhere',
 'fit': 'fit',
 'separate

In [ ]:
impfOld.loc[impfOld.era>=5, "lemma"] = impfOld.loc[impfOld.era>=5, "V1"].str.lower().map(impfDicts)

In [ ]:
impfOld.loc[impfOld.V1 == "=", "lemma"] = "NA" 
impfOld.loc[6897, "lemma"] = "suppose"

In [ ]:
impfOld.loc[impfOld.lemma.isna()]

In [ ]:
impfOld.to_csv("impfLemmatized.csv")

In [106]:
impf4 = pd.read_csv("impfEL4.csv", usecols=[1,2,3,4,5])
impfOld = pd.read_csv("impfLemmatized.csv")

impf4 = impf4.rename(columns = {"0": "lemma"})

In [93]:
impf4.loc[impf4.lemma.isna() & (impf4.era<5)]

,V1,fn,era,lemma,notes
459,nach+gi+d,cmancriw-1-m1,1,NaN,***p. 37
461,nach+gi,cmancriw-1-m1,1,NaN,***p. 37
502,bilumpen,cmancriw-1-m1,1,NaN,"? Means ""reveal"", I think"
554,'+teostrit,cmancriw-1-m1,1,NaN,??????
637,'+toncke,cmancriw-1-m1,1,NaN,??????
684,dach+ge,cmancriw-1-m1,1,NaN,NaN
703,tetre+d,cmancriw-1-m1,1,NaN,NaN
750,cheafli+d,cmancriw-1-m1,1,NaN,??????
1057,$+tur+de,cmancriw-1-m1,1,NaN,NaN
1107,$honli,cmancriw-1-m1,1,NaN,NaN


In [107]:
impfcors1 = {t[0]: t[1] for t in impf4.loc[((impf4.era<5) & impf4.lemma.notna()), ["V1", "lemma"]].values.tolist()}
impfcors2 = {t[0]: t[1] for t in impfOld.loc[((impfOld.era<5) & impfOld.lemma.notna()), ["V1", "lemma"]].values.tolist()}

impfcors3 = {}
impfcors3.update(impfcors2)
impfcors3.update(impfcors1)

In [108]:
impfOld.loc[impfOld.era<5, "lemma"] = impfOld.loc[impfOld.era<5, "V1"].str.lower().map(impfcors3)

In [109]:
impfOld.loc[(impfOld.era<5) & impfOld.lemma.isna()]

,Unnamed: 0,V1,fn,era,lemma
47020,47020,brenk+t,cmaelr3-m23,2,NaN
47054,47054,vele+t,cmaelr3-m23,2,NaN
47063,47063,flikt,cmaelr3-m23,2,NaN
47093,47093,sechy+t,cmaelr3-m23,2,NaN
47139,47139,say+gt,cmaelr3-m23,2,NaN
47145,47145,vieledest,cmaelr3-m23,2,NaN
47159,47159,$by+genkest,cmaelr3-m23,2,NaN
47171,47171,traw+t,cmaelr3-m23,2,NaN
47179,47179,for+gyte,cmaelr3-m23,2,NaN
47226,47226,to-tery+t,cmaelr3-m23,2,NaN


In [126]:
array1 = [tuple(t) for t in impf4.loc[impf4.era<5, ["V1", "lemma"]].values]
array2 = [tuple(t) for t in impfOld.loc[impfOld.era<5, ["V1", "lemma"]].values]

set1 = {t for t in array1}
set2 = {t for t in array2}

set1 - set2

{('$Denyest', 'denien'),
 ('$Spit', 'spitten'),
 ('$Undirstondest', 'understonden'),
 ('$Wenest', 'wenen'),
 ('+Gif', nan),
 ('+Tench', 'thinken'),
 ('+gernes', nan),
 ('+gernys', nan),
 ('+tarf', nan),
 ('Abbot', 'BAD'),
 ('BEGINNI+T', 'biginnen'),
 ('BIGINNI+T', 'biginnen'),
 ('BIGINNY+T', 'biginnen'),
 ('BIGYNNI+T', 'biginnen'),
 ('BIGYNNY+T', 'biginnen'),
 ('BYGINNI+T', 'biginnen'),
 ('BYGYNNI+T', 'biginnen'),
 ('Begynn', 'biginnen'),
 ('Bi+geote+d', 'biyeten'),
 ('Bitacnenn', 'bitoknen'),
 ('Comenn', 'comen'),
 ('Culche', 'gulchen'),
 ('De-failen', 'defailen'),
 ('Defailen', 'defailen'),
 ('Engendyrs', 'engendren'),
 ('Englishe', 'Englishen'),
 ('Enioyne', 'enjoinen'),
 ('Enjoyne', 'enjoinen'),
 ('Enprides', 'empriden'),
 ('Enquere', 'enqueren'),
 ('Erne+t+t', 'rennen'),
 ('Flittenn', 'flitten'),
 ('Forr+gete+t+t', 'foryeten'),
 ('Forrblende+t+t', 'forblenden'),
 ('Forrdo+t', 'fordon'),
 ('Forrho+ghe+t+t', 'forhouen'),
 ('Forrho+ghenn', 'forhouen'),
 ('Forrsake+t+t', 'forsaken'),


In [136]:
array3 = [tuple(t) for t in impf4.loc[impf4.era<5, ["V1", "lemma"]].values if isinstance(t[1], str)]

set3 = {t for t in array3}

missed = set3 - set2
mtokens = [t[0] for t in missed]
mdict = {t[0]: t[1] for t in missed}

In [143]:
impfOld2 = impfOld
impfOld2.loc[impfOld2.V1.isin(mtokens) & (impfOld2.era<5) & impfOld2.lemma.isna(), "lemma"] = impfOld2.loc[impfOld2.V1.isin(mtokens) & (impfOld2.era<5) & impfOld2.lemma.isna(), "V1"].map(mdict)
impfOld2.loc[impfOld2.V1.isin(mtokens) & (impfOld2.era<5)]

,Unnamed: 0,V1,fn,era,lemma
47243,47243,Sykst,cmaelr3-m23,2,siken
47376,47376,Sykst,cmaelr3-m23,2,siken
48103,48103,$Spit,cmancriw-1-m1,1,spitten
48450,48450,Galie+d,cmancriw-1-m1,1,galen
48536,48536,Temen,cmancriw-1-m1,1,temen
48829,48829,Pruuie+d,cmancriw-1-m1,1,preven
48831,48831,Mutle+d,cmancriw-1-m1,1,muchelen
48993,48993,Culche,cmancriw-1-m1,1,gulchen
49244,49244,Muchele+d,cmancriw-1-m1,1,muchelen
49681,49681,Mudle+d,cmancriw-1-m1,1,muchelen


In [151]:
impfOld2.to_csv("impfLemmatizedV2.csv")

In [ ]:
pde = pd.read_csv("lemmalist.csv", usecols=[1,2,3])
pde.head()

pdeDict = {t[0]: t[1] for t in pde[1:].values.tolist() if isinstance(t[1], str)}
pdeDict

impfPDE = impfOld2
impfPDE['PDElem'] = impfPDE.loc[(impfPDE.era<5), "lemma"].map(pdeDict)
impfPDE.loc[impfPDE.era<5].head()

impfPDE.loc[impfPDE.era>4, 'PDElem'] = impfPDE.loc[impfPDE.era>4, 'lemma']
impfPDE.head()

impfPDE[["V1", "lemma", "PDElem", "era"]].to_csv("impfLemPDE.csv")

In [231]:
progPDE = pd.read_csv("progLemmatized.csv", usecols=[1,2,3,4,5,6])

,vbg,beform,fn,era,lemma,notes
0,fleotinde,(BEP beon),cmancriw-1-m1,1,fleten,NaN
1,sturinde,(BEI beo+d),cmancriw-1-m1,1,stiren,NaN
2,fleoinde,(BEI beo+d),cmancriw-1-m1,1,flien,NaN
3,herinde,(BEP is),cmancriw-1-m1,1,heren,NaN
4,halewinde,(BEP is),cmancriw-1-m1,1,halwen,NaN


In [ ]:
progPDE['PDElem'] = progPDE.loc[(progPDE.era<5), "lemma"].map(pdeDict)
progPDE.loc[progPDE.era<5].head()

progPDE.loc[progPDE.era>4, 'PDElem'] = progPDE.loc[progPDE.era>4, 'lemma']

In [294]:
progPDE[["vbg","lemma","PDElem","era", "fn", "beform", "notes"]].to_csv("progMasterV1.csv")

In [293]:
progPDE.loc[74, "PDElem"] = "wear"
progPDE.loc[76, "PDElem"] = "overwhelm"

In [264]:
newlemmas = pd.read_csv("lemmas-to-add-041619.csv", usecols=[1,2,3,4,5], index_col=0)
newlemmas.head()

,V1,lemma,PDElem,era
X,,,,
48161,nach+gi+d,neighen,neigh,1
48163,nach+gi,neighen,neigh,1
48204,bilumpen,bilimpen,belimp,1
48256,+teostrit,thestren,thester,1
48339,+toncke,thanken,thank,1


In [265]:
impfPDE = pd.read_csv("impfLemPDE.csv", usecols=[1,2,3,4])
impfPDE.head()

,V1,lemma,PDElem,era
0,opens,open,open,9
1,consider,consider,consider,9
2,remains,remain,remain,9
3,takes,take,take,9
4,depends,depend,depend,9


In [266]:
impfPDE.update(newlemmas)

In [270]:
impfPDE.head()

,V1,lemma,PDElem,era
0,opens,open,open,9.0
1,consider,consider,consider,9.0
2,remains,remain,remain,9.0
3,takes,take,take,9.0
4,depends,depend,depend,9.0


In [281]:
impfPDE.to_csv("impfPDEv2.csv")

In [279]:
impfPDE.loc[impfPDE.PDElem=="comes"]

,V1,lemma,PDElem,era


In [284]:
progPDE.loc[progPDE.PDElem=="know"]

,vbg,beform,fn,era,lemma,notes,PDElem
175,knowyng,(BEP is),cmboeth-m3,3,knouen,NaN,know
671,knowing,(BEP is),osborne-e3-h,7,know,NaN,know


In [313]:
impfO = pd.read_csv("impfLemmatizedV2.csv", usecols=[2,3,4,5])
impfO.head()

,V1,fn,era,lemma
0,opens,adams-1787-2,9,open
1,consider,adams-1787-2,9,consider
2,remains,adams-1787-2,9,remain
3,takes,adams-1787-2,9,take
4,depends,adams-1787-2,9,depend


In [314]:
impfO.loc[impfO.lemma.isna()]

,V1,fn,era,lemma
11953,=,dalton-1808-2,9,NaN
11998,=,dalton-1808-2,9,NaN
11999,=,dalton-1808-2,9,NaN
12012,=,dalton-1808-2,9,NaN
12013,=,dalton-1808-2,9,NaN
12033,=,dalton-1808-2,9,NaN
12034,=,dalton-1808-2,9,NaN
14367,ate,erv-new-1881-1,10,NaN
14981,madest,erv-new-1881-2,10,NaN
14992,crucified,erv-new-1881-2,10,NaN


In [315]:
lemDict = {t[0]: t[1] for t in impfPDE.values.tolist() if isinstance(t[1], str)}
lemDict

{'opens': 'open',
 'consider': 'consider',
 'remains': 'remain',
 'takes': 'take',
 'depends': 'depend',
 'nourish': 'nourish',
 'maintain': 'maintain',
 'carry': 'carry',
 'obtains': 'obtain',
 'appears': 'appear',
 'covers': 'cover',
 'says': 'say',
 'meets': 'meet',
 'assume': 'assume',
 'asserts': 'assert',
 'makes': 'make',
 'suppose': 'suppose',
 'appear': 'appear',
 'lie': 'lie',
 'learn': 'learn',
 'regenerates': 'regenerate',
 'preserves': 'preserves',
 'prevents': 'prevent',
 'run': 'run',
 'dries': 'dry',
 'shrink': 'shrink',
 'disappear': 'disappear',
 'form': 'form',
 'find': 'find',
 'contracts': 'contract',
 'gets': 'get',
 'suffices': 'suffice',
 'arrives': 'arrive',
 'grow': 'grow',
 'expands': 'expand',
 'rushes': 'rush',
 'varies': 'vary',
 'exists': 'exist',
 'forms': 'form',
 'adheres': 'adhere',
 'falls': 'fall',
 'consists': 'consist',
 'derives': 'derive',
 'depend': 'depend',
 'thinks': 'think',
 'leaves': 'leave',
 'adhere': 'adhere',
 'fit': 'fit',
 'separate

In [316]:
impfO['lemma'] = impfO.V1.map(lemDict)

In [317]:
pdeDict = {t[1]: t[2] for t in impfPDE.values.tolist() if isinstance(t[1], str)}
pdeDict

{'open': 'open',
 'consider': 'consider',
 'remain': 'remain',
 'take': 'take',
 'depend': 'depend',
 'nourish': 'nourish',
 'maintain': 'maintain',
 'carry': 'carry',
 'obtain': 'obtain',
 'appear': 'appear',
 'cover': 'cover',
 'say': 'say',
 'meet': 'meet',
 'assume': 'assume',
 'assert': 'assert',
 'make': 'make',
 'suppose': 'suppose',
 'lie': 'lie',
 'learn': 'learn',
 'regenerate': 'regenerate',
 'preserves': 'preserves',
 'prevent': 'prevent',
 'run': 'run',
 'dry': 'dry',
 'shrink': 'shrink',
 'disappear': 'disappear',
 'form': 'form',
 'find': 'find',
 'contract': 'contract',
 'get': 'get',
 'suffice': 'suffice',
 'arrive': 'arrive',
 'grow': 'grow',
 'expand': 'expand',
 'rush': 'rush',
 'vary': 'vary',
 'exist': 'exist',
 'adhere': 'adhere',
 'fall': 'fall',
 'consist': 'consist',
 'derive': 'derive',
 'think': 'think',
 'leave': 'leave',
 'fit': 'fit',
 'separate': 'separate',
 'differ': 'differ',
 'hold': 'hold',
 'dilate': 'dilate',
 'become': 'become',
 'call': 'call',


In [318]:
impfO['PDElem'] = impfO.lemma.map(pdeDict)
impfO.loc[impfO.PDElem.isna()]

,V1,fn,era,lemma,PDElem
11953,=,dalton-1808-2,9,NaN,NaN
11998,=,dalton-1808-2,9,NaN,NaN
11999,=,dalton-1808-2,9,NaN,NaN
12012,=,dalton-1808-2,9,NaN,NaN
12013,=,dalton-1808-2,9,NaN,NaN
12033,=,dalton-1808-2,9,NaN,NaN
12034,=,dalton-1808-2,9,NaN,NaN
14367,ate,erv-new-1881-1,10,x,NaN
14981,madest,erv-new-1881-2,10,x,NaN
14992,crucified,erv-new-1881-2,10,x,NaN


In [326]:
impfO.loc[impfO.lemma=="comes", "lemma"] = "come"
impfO.loc[impfO.lemma=="comes"]

,V1,fn,era,lemma,PDElem


In [327]:
impfO[["V1","lemma","PDElem","era", "fn"]].to_csv("impfMasterV2.csv")